Copyright 2024 Google. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataframe = pd.read_csv('elo.csv')
dataframe

,method,elo,p99Low,p99Hi,bpp,label
0,jpegli-q55-yuv444,1616.2178,1570.1244,1662.3113,0.896926,jpegli
1,jpegli-q60-yuv444,1656.4395,1611.1019,1701.7771,0.963380,jpegli
2,jpegli-q65-yuv420,1600.8348,1554.3654,1647.3044,0.874811,jpegli-420
3,jpegli-q65-yuv444,1738.0724,1695.7542,1780.3906,1.029448,jpegli
4,jpegli-q70-yuv420,1692.4553,1648.4811,1736.4296,0.953669,jpegli-420
5,jpegli-q70-yuv444,1864.5178,1824.3069,1904.7288,1.123597,jpegli
6,jpegli-q75-yuv420,1823.9264,1782.0206,1865.8322,1.051037,jpegli-420
7,jpegli-q75-yuv444,2022.0585,1980.5918,2063.5254,1.227468,jpegli
8,jpegli-q80-yuv420,1980.5387,1935.3760,2025.7014,1.176039,jpegli-420
9,jpegli-q80-yuv444,2139.4082,2096.1480,2182.6687,1.374517,jpegli


In [ ]:
data = dataframe.values
methods = np.unique(data[:, -1])
elos = {
    method: data[data[:, -1] == method][:, 1].astype(np.float32)
    for method in methods
}
bitrates = {
    method: data[data[:, -1] == method][:, 4].astype(np.float32)
    for method in methods
}
turbo_names = data[data[:, -1] == 'libjpeg-turbo'][:, 0]
elos_to_compare = []
turbo_equivalent_qualities = []
for index, elo in enumerate(elos['libjpeg-turbo']):
  if (
      elo >= min(elos['jpegli'])
      and elo >= min(elos['mozjpeg'])
      and elo <= max(elos['jpegli'])
      and elo <= max(elos['mozjpeg'])
  ):
    elos_to_compare.append(elo)
    turbo_equivalent_qualities.append(turbo_names[index])


def bitrate_at_elo(method, elo):
  return np.interp(elo, elos[method], bitrates[method])


def elo_at_bitrate(method, bitrate):
  return np.interp(bitrate, bitrates[method], elos[method])


print(
    f'libjpeg_turbo_equivalent_quality,elo,libjpeg_turbo_bitrate,mozjpeg_bitrate,jpegli_bitrate'
)
for i in range(len(elos_to_compare)):
  elo = elos_to_compare[i]
  name = turbo_equivalent_qualities[i]
  print(
      f"{name},{elo},{bitrate_at_elo('libjpeg-turbo', elo)},{bitrate_at_elo('mozjpeg', elo)},{bitrate_at_elo('jpegli', elo)}"
  )

libjpeg_turbo_equivalent_quality,elo,libjpeg_turbo_bitrate,mozjpeg_bitrate,jpegli_bitrate
libjpeg-turbo-q70-yuv420,1685.1304931640625,1.1296766996383667,0.940491951974611,0.986600255931497
libjpeg-turbo-q75-yuv420,1757.050537109375,1.2395873069763184,1.0206446804441114,1.0435789756588576
libjpeg-turbo-q80-yuv422,1989.6468505859375,1.5401406288146973,1.3985910069922816,1.2060982872396786
libjpeg-turbo-q85-yuv422,2150.5595703125,1.8047833442687988,1.6460170723784693,1.3902699242484242
libjpeg-turbo-q90-yuv444,2392.5341796875,2.622631549835205,2.6262534131392052,1.8477547233536156
libjpeg-turbo-q95-yuv444,2608.0166015625,3.774329900741577,3.5005639449528534,2.6738307925538596


In [ ]:
turbo_bitrate_at_elo_2_1 = elo_at_bitrate('libjpeg-turbo', 2.1)
print(f'{turbo_bitrate_at_elo_2_1=}')
print(f'{bitrate_at_elo("jpegli", turbo_bitrate_at_elo_2_1)=}')

turbo_bitrate_at_elo_2_1=2237.9045500597854
bitrate_at_elo("jpegli", turbo_bitrate_at_elo_2_1)=1.5136595319075263
